In [ ]:
pip install ultralytics optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 7.4 MB/s eta 0:00:00


In [ ]:
import torch
import yaml
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import numpy as np
from pathlib import Path
from typing import Tuple, Optional, Dict, Any, List

class YOLOImagePreprocessor:
    """
    Image preprocessor for YOLO training.
    """
    def __init__(self, target_size: Tuple[int, int] = (640, 640)):
        self.target_size = target_size
        self.padding_color = (114, 114, 114)  # Default gray padding

    def process_image(self, image_path: str) -> Optional[np.ndarray]:
        """
        Process a single image with better error handling and preprocessing.
        """
        try:
            image = cv2.imread(str(image_path))
            if image is None:
                raise ValueError(f"Failed to load image: {image_path}")

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Find non-black region
            mask = np.any(image != [0, 0, 0], axis=2)
            rows = np.any(mask, axis=1)
            cols = np.any(mask, axis=0)

            if not np.any(rows) or not np.any(cols):
                raise ValueError(f"Image {image_path} appears to be completely black")

            # Get the non-black region boundaries
            y1, y2 = np.where(rows)[0][[0, -1]]
            x1, x2 = np.where(cols)[0][[0, -1]]

            # Crop the image
            cropped = image[y1:y2+1, x1:x2+1]

            # Preserve aspect ratio
            h, w = cropped.shape[:2]
            scale = min(self.target_size[0]/w, self.target_size[1]/h)
            new_w = int(w * scale)
            new_h = int(h * scale)

            # Resize with maintained aspect ratio
            resized = cv2.resize(cropped, (new_w, new_h), interpolation=cv2.INTER_AREA)

            # Create canvas with padding
            canvas = np.ones((self.target_size[0], self.target_size[1], 3),
                           dtype=np.uint8) * self.padding_color[0]

            # Calculate padding
            pad_x = (self.target_size[1] - new_w) // 2
            pad_y = (self.target_size[0] - new_h) // 2

            # Place image on canvas
            canvas[pad_y:pad_y+new_h, pad_x:pad_x+new_w] = resized

            return canvas

        except Exception as e:
            print(f"Error preprocessing image {image_path}: {str(e)}")
            return None

    def process_dataset(self, dataset_path: Path, output_path: Path) -> Dict[str, Dict[str, int]]:
        """
        Process entire dataset and track transformations.
        """
        output_path.mkdir(exist_ok=True, parents=True)
        transformations = {}

        for img_path in dataset_path.glob('*.jpg'):
            try:
                processed = self.process_image(str(img_path))
                if processed is not None:
                    output_img_path = output_path / img_path.name
                    cv2.imwrite(str(output_img_path), cv2.cvtColor(processed, cv2.COLOR_RGB2BGR))

                    transformations[img_path.name] = {
                        'original_size': tuple(cv2.imread(str(img_path)).shape[:2]),
                        'processed_size': processed.shape[:2]
                    }
            except Exception as e:
                print(f"Error processing {img_path}: {str(e)}")

        return transformations

class YOLOTrainer:
    """
    Simplified YOLO trainer with fixed hyperparameters.
    """
    def __init__(self,
                 dataset_path: str,
                 model_path: str = 'yolov8n.pt',
                 save_dir: str = 'saved_models'):
        self.dataset_path = dataset_path
        self.model_path = model_path
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True, parents=True)
        self.preprocessor = YOLOImagePreprocessor(target_size=(640, 640))

        # Load and validate dataset configuration
        with open(dataset_path, 'r') as f:
            self.dataset_config = yaml.safe_load(f)

        print("Dataset Configuration:")
        print(yaml.dump(self.dataset_config, default_flow_style=False))

        # Fixed hyperparameters (optimized values)
        self.hyperparameters = {
            'lr0': 0.01,
            'lrf': 0.05,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3,
            'warmup_momentum': 0.8,
            'batch_size': 16,
            'mosaic': 0.5
        }

    def prepare_dataset(self) -> Tuple[Dict[str, Dict[str, int]], Dict[str, Dict[str, int]]]:
        """
        Prepare and validate the dataset before training.
        """
        train_path = Path(self.dataset_config.get('train', ''))
        val_path = Path(self.dataset_config.get('val', ''))

        # Process training and validation datasets
        train_transforms = self.preprocessor.process_dataset(
            train_path,
            train_path.parent / 'processed_train'
        )
        val_transforms = self.preprocessor.process_dataset(
            val_path,
            val_path.parent / 'processed_val'
        )

        return train_transforms, val_transforms

    def train_model(self,
                   save_name: str = 'trained_model',
                   epochs: int = 100) -> Dict[str, Any]:
        """
        Train model with fixed hyperparameters.
        """
        model = YOLO(self.model_path)

        save_dir = self.save_dir / save_name
        save_dir.mkdir(exist_ok=True)

        # Training arguments
        args = {
            'data': self.dataset_path,
            'epochs': epochs,
            'batch': self.hyperparameters['batch_size'],
            'lr0': self.hyperparameters['lr0'],
            'lrf': self.hyperparameters['lrf'],
            'momentum': self.hyperparameters['momentum'],
            'weight_decay': self.hyperparameters['weight_decay'],
            'warmup_epochs': self.hyperparameters['warmup_epochs'],
            'warmup_momentum': self.hyperparameters['warmup_momentum'],
            'mosaic': self.hyperparameters['mosaic'],
            'project': str(self.save_dir),
            'name': save_name,
            'exist_ok': True
        }

        # Train the model
        results = model.train(**args)

        # Save the model
        model_save_path = save_dir / 'weights'
        model_save_path.mkdir(exist_ok=True)

        # Save in different formats
        model.save(str(model_save_path / 'best.pt'))
        model.save(str(model_save_path / 'last.pt'))

        # Export to ONNX
        try:
            model.export(format='onnx', save_dir=model_save_path)
        except Exception as e:
            print(f"ONNX export failed: {str(e)}")

        # Save results
        with open(save_dir / 'training_results.yaml', 'w') as f:
            yaml.dump(results.results_dict, f)

        return results.results_dict

    def plot_training_history(self, results: Dict[str, Any]) -> plt.Figure:
        """
        Plot training metrics and save the figure.
        """
        plt.figure(figsize=(15, 10))

        # Plot training and validation loss
        plt.subplot(2, 2, 1)
        plt.plot(results['train/loss'], label='Training Loss')
        plt.plot(results['val/loss'], label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # Plot mAP metrics
        plt.subplot(2, 2, 2)
        plt.plot(results['metrics/mAP50-95(B)'], label='mAP50-95')
        plt.plot(results['metrics/mAP50(B)'], label='mAP50')
        plt.title('Mean Average Precision')
        plt.xlabel('Epoch')
        plt.ylabel('mAP')
        plt.legend()

        # Plot learning rate
        plt.subplot(2, 2, 3)
        plt.plot(results['train/lr0'], label='Learning Rate')
        plt.title('Learning Rate Schedule')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.yscale('log')

        # Plot batch size
        plt.subplot(2, 2, 4)
        plt.plot(results['train/batch_size'], label='Batch Size')
        plt.title('Batch Size')
        plt.xlabel('Epoch')
        plt.ylabel('Batch Size')

        plt.tight_layout()
        return plt.gcf()

def main():
    """
    Main execution function.
    """
    # Configuration
    dataset_path = '/content/Deep-learning-2/data.yaml'
    save_dir = '/content/drive/MyDrive/yolo_v8'
    epochs = 100

    # Initialize trainer
    trainer = YOLOTrainer(
        dataset_path=dataset_path,
        model_path='yolov8n.pt',
        save_dir=save_dir
    )

    # Train model
    print("\nTraining model...")
    results = trainer.train_model(
        save_name='trained_model',
        epochs=epochs
    )

    # Plot and save training history
    print("\nGenerating training plots...")
    fig = trainer.plot_training_history(results)
    fig.savefig(os.path.join(save_dir, 'trained_model', 'training_history.png'))
    plt.close(fig)

    print("\nTraining completed successfully!")
    print(f"Model and results saved in: {save_dir}/trained_model/")

if __name__ == '__main__':
    main()

Dataset Configuration:
names:
- car
- cars
- hair clip
- key chain
- leaf
- leafs
- lid
- pen
- pencil
nc: 9
roboflow:
  license: CC BY 4.0
  project: deep-learning-khbes
  url: https://universe.roboflow.com/deep-learning-mvdfy/deep-learning-khbes/dataset/2
  version: 2
  workspace: deep-learning-mvdfy
test: ../test/images
train: ../train/images
val: ../valid/images


Training model...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Deep-learning-2/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolo_v8, name=trained_model, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mas

train: Scanning /content/Deep-learning-2/train/labels.cache... 97 images, 6 backgrounds, 0 corrupt: 100%|██████████| 97/97 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 5, len(boxes) = 124. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/Deep-learning-2/valid/labels.cache... 26 images, 3 backgrounds, 0 corrupt: 100%|██████████| 26/26 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/yolo_v8/trained_model/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/yolo_v8/trained_model
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         26         36      0.814     0.0934      0.159     0.0813



100 epochs completed in 0.083 hours.
Optimizer stripped from /content/drive/MyDrive/yolo_v8/trained_model/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/yolo_v8/trained_model/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/yolo_v8/trained_model/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.01it/s]


                   all         26         36      0.753     0.0678      0.247      0.151
                  cars          2          2          1          0      0.586      0.401
             hair clip          3          3          1          0      0.356      0.345
             key chain          1          1          1          0    0.00403   0.000403
                  leaf          8         13      0.199      0.308      0.112     0.0384
                 leafs          6          6     0.0724      0.167      0.122     0.0822
                   pen          4          4          1          0       0.25      0.137
                pencil          3          7          1          0      0.299     0.0562
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_v8/trained_model


TypeError: 'list' object is not callable

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="JdViCZkE1bnhWcRhwrM1")
project = rf.workspace("deep-learning-mvdfy").project("deep-learning-khbes")
version = project.version(2)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Deep-learning-2 in yolov8:: 100%|██████████| 255/255 [00:00<00:00, 4353.75it/s]


In [14]:
import torch
import yaml
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import numpy as np
from pathlib import Path
from typing import Tuple, Optional, Dict, Any, List

class YOLOImagePreprocessor:
    """
    Image preprocessor for YOLO training.
    """
    def __init__(self, target_size: Tuple[int, int] = (640, 640)):
        self.target_size = target_size
        self.padding_color = (114, 114, 114)  # Default gray padding

    def process_image(self, image_path: str) -> Optional[np.ndarray]:
        """
        Process a single image with better error handling and preprocessing.
        """
        try:
            image = cv2.imread(str(image_path))
            if image is None:
                raise ValueError(f"Failed to load image: {image_path}")

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Find non-black region
            mask = np.any(image != [0, 0, 0], axis=2)
            rows = np.any(mask, axis=1)
            cols = np.any(mask, axis=0)

            if not np.any(rows) or not np.any(cols):
                raise ValueError(f"Image {image_path} appears to be completely black")

            # Get the non-black region boundaries
            y1, y2 = np.where(rows)[0][[0, -1]]
            x1, x2 = np.where(cols)[0][[0, -1]]

            # Crop the image
            cropped = image[y1:y2+1, x1:x2+1]

            # Preserve aspect ratio
            h, w = cropped.shape[:2]
            scale = min(self.target_size[0]/w, self.target_size[1]/h)
            new_w = int(w * scale)
            new_h = int(h * scale)

            # Resize with maintained aspect ratio
            resized = cv2.resize(cropped, (new_w, new_h), interpolation=cv2.INTER_AREA)

            # Create canvas with padding
            canvas = np.ones((self.target_size[0], self.target_size[1], 3),
                           dtype=np.uint8) * self.padding_color[0]

            # Calculate padding
            pad_x = (self.target_size[1] - new_w) // 2
            pad_y = (self.target_size[0] - new_h) // 2

            # Place image on canvas
            canvas[pad_y:pad_y+new_h, pad_x:pad_x+new_w] = resized

            return canvas

        except Exception as e:
            print(f"Error preprocessing image {image_path}: {str(e)}")
            return None

    def process_dataset(self, dataset_path: Path, output_path: Path) -> Dict[str, Dict[str, int]]:
        """
        Process entire dataset and track transformations.
        """
        output_path.mkdir(exist_ok=True, parents=True)
        transformations = {}

        for img_path in dataset_path.glob('*.jpg'):
            try:
                processed = self.process_image(str(img_path))
                if processed is not None:
                    output_img_path = output_path / img_path.name
                    cv2.imwrite(str(output_img_path), cv2.cvtColor(processed, cv2.COLOR_RGB2BGR))

                    transformations[img_path.name] = {
                        'original_size': tuple(cv2.imread(str(img_path)).shape[:2]),
                        'processed_size': processed.shape[:2]
                    }
            except Exception as e:
                print(f"Error processing {img_path}: {str(e)}")

        return transformations

class YOLOTrainer:
    """
    Simplified YOLO trainer with fixed hyperparameters.
    """
    def __init__(self,
                 dataset_path: str,
                 model_path: str = 'yolov8n.pt',
                 save_dir: str = 'saved_models'):
        self.dataset_path = dataset_path
        self.model_path = model_path
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True, parents=True)
        self.preprocessor = YOLOImagePreprocessor(target_size=(640, 640))

        # Load and validate dataset configuration
        with open(dataset_path, 'r') as f:
            self.dataset_config = yaml.safe_load(f)

        print("Dataset Configuration:")
        print(yaml.dump(self.dataset_config, default_flow_style=False))

        # Fixed hyperparameters (optimized values)
        self.hyperparameters = {
            'lr0': 0.01,
            'lrf': 0.05,
            'momentum': 0.937,
            'weight_decay': 0.0005,
            'warmup_epochs': 3,
            'warmup_momentum': 0.8,
            'batch_size': 16,
            'mosaic': 0.5
        }

    def prepare_dataset(self) -> Tuple[Dict[str, Dict[str, int]], Dict[str, Dict[str, int]]]:
        """
        Prepare and validate the dataset before training.
        """
        train_path = Path(self.dataset_config.get('train', ''))
        val_path = Path(self.dataset_config.get('val', ''))

        # Process training and validation datasets
        train_transforms = self.preprocessor.process_dataset(
            train_path,
            train_path.parent / 'processed_train'
        )
        val_transforms = self.preprocessor.process_dataset(
            val_path,
            val_path.parent / 'processed_val'
        )

        return train_transforms, val_transforms

    def train_model(self,
                   save_name: str = 'trained_model',
                   epochs: int = 100) -> Dict[str, Any]:
        """
        Train model with fixed hyperparameters.
        """
        model = YOLO(self.model_path)

        save_dir = self.save_dir / save_name
        save_dir.mkdir(exist_ok=True)

        # Training arguments
        args = {
            'data': self.dataset_path,
            'epochs': epochs,
            'batch': self.hyperparameters['batch_size'],
            'lr0': self.hyperparameters['lr0'],
            'lrf': self.hyperparameters['lrf'],
            'momentum': self.hyperparameters['momentum'],
            'weight_decay': self.hyperparameters['weight_decay'],
            'warmup_epochs': self.hyperparameters['warmup_epochs'],
            'warmup_momentum': self.hyperparameters['warmup_momentum'],
            'mosaic': self.hyperparameters['mosaic'],
            'project': str(self.save_dir),
            'name': save_name,
            'exist_ok': True
        }

        # Train the model
        results = model.train(**args)

        # Save the model
        model_save_path = save_dir / 'weights'
        model_save_path.mkdir(exist_ok=True)

        # Save in different formats
        model.save(str(model_save_path / 'best.pt'))
        model.save(str(model_save_path / 'last.pt'))

        # Export to ONNX
        try:
            model.export(format='onnx', save_dir=model_save_path)
        except Exception as e:
            print(f"ONNX export failed: {str(e)}")

        # Save results
        with open(save_dir / 'training_results.yaml', 'w') as f:
            yaml.dump(results.results_dict, f)

        return results.results_dict

    # def plot_training_history(self, results: Dict[str, Any]) -> plt.Figure:
    #     """
    #     Plot training metrics and save the figure.
    #     """
    #     plt.figure(figsize=(15, 10))



        # # Plot mAP metrics
        # plt.subplot(2, 2, 2)
        # plt.plot(results['metrics/mAP50-95(B)'], label='mAP50-95')
        # plt.plot(results['metrics/mAP50(B)'], label='mAP50')
        # plt.title('Mean Average Precision')
        # plt.xlabel('Epoch')
        # plt.ylabel('mAP')
        # plt.legend()

        # Plot learning rate
        # plt.subplot(2, 2, 3)
        # plt.plot(results['train/lr0'], label='Learning Rate')
        # plt.title('Learning Rate Schedule')
        # plt.xlabel('Epoch')
        # plt.ylabel('Learning Rate')
        # plt.yscale('log')

        # # Plot batch size
        # plt.subplot(2, 2, 4)
        # plt.plot(results['train/batch_size'], label='Batch Size')
        # plt.title('Batch Size')
        # plt.xlabel('Epoch')
        # plt.ylabel('Batch Size')

        # plt.tight_layout()
        # return plt.gcf()

def main():
    """
    Main execution function.
    """
    # Configuration
    dataset_path = '/content/Deep-learning-2/data.yaml'
    save_dir = '/content/drive/MyDrive/yolo_1'
    epochs = 250

    # Initialize trainer
    trainer = YOLOTrainer(
        dataset_path=dataset_path,
        model_path='yolov8n.pt',
        save_dir=save_dir
    )

    # Train model
    print("\nTraining model...")
    results = trainer.train_model(
        save_name='trained_model',
        epochs=epochs
    )

    # # Plot and save training history
    # print("\nGenerating training plots...")
    # fig = trainer.plot_training_history(results)
    # fig.savefig(os.path.join(save_dir, 'trained_model', 'training_history.png'))
    # plt.close(fig)

    print("\nTraining completed successfully!")
    print(f"Model and results saved in: {save_dir}/trained_model/")

if __name__ == '__main__':
    main()

Dataset Configuration:
names:
- car
- cars
- hair clip
- key chain
- leaf
- leafs
- lid
- pen
- pencil
nc: 9
roboflow:
  license: CC BY 4.0
  project: deep-learning-khbes
  url: https://universe.roboflow.com/deep-learning-mvdfy/deep-learning-khbes/dataset/2
  version: 2
  workspace: deep-learning-mvdfy
test: ../test/images
train: ../train/images
val: ../valid/images


Training model...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Deep-learning-2/data.yaml, epochs=250, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolo_1, name=trained_model, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask

train: Scanning /content/Deep-learning-2/train/labels.cache... 97 images, 6 backgrounds, 0 corrupt: 100%|██████████| 97/97 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 5, len(boxes) = 124. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/Deep-learning-2/valid/labels.cache... 26 images, 3 backgrounds, 0 corrupt: 100%|██████████| 26/26 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/yolo_1/trained_model/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/yolo_1/trained_model
Starting training for 250 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    214/250      4.15G      0.386     0.9113     0.9969          1        640: 100%|██████████| 7/7 [00:01<00:00,  4.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.10it/s]

                   all         26         36      0.514      0.152      0.247      0.162
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 114, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



214 epochs completed in 0.186 hours.
Optimizer stripped from /content/drive/MyDrive/yolo_1/trained_model/weights/last.pt, 6.3MB
Optimizer stripped from /content/drive/MyDrive/yolo_1/trained_model/weights/best.pt, 6.3MB

Validating /content/drive/MyDrive/yolo_1/trained_model/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.65it/s]


                   all         26         36        0.7      0.274      0.368        0.2
                  cars          2          2          1      0.941      0.995      0.697
             hair clip          3          3      0.437      0.333      0.641      0.476
             key chain          1          1          1          0      0.497     0.0498
                  leaf          8         13       0.26      0.308      0.224     0.0784
                 leafs          6          6      0.202      0.333      0.221      0.102
                   pen          4          4          1          0          0          0
                pencil          3          7          1          0          0          0
Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_1/trained_model
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradien

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
